Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [7]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [79]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  learning_rate = 0.5
  penalty_constant = 1/float(1500)
  logits = tf.matmul(tf_train_dataset, weights) + biases #return (train_subset, num_labels)
  #print(logits.get_shape())
  ##penalty= penalty_constant*1/2*(weights)**2 == nn.l2_loss(t) #returns (image_size**2, image_size**2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  l2 = loss + penalty_constant*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases))
  #loss and l2 are not defined until sess.run()
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l2)

  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [80]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:

  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):

    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))

      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.841317
Training accuracy: 13.7%
Validation accuracy: 16.7%
Loss at step 100: 2.301761
Training accuracy: 71.5%
Validation accuracy: 70.4%
Loss at step 200: 1.771502
Training accuracy: 74.7%
Validation accuracy: 73.3%
Loss at step 300: 1.485755
Training accuracy: 76.3%
Validation accuracy: 74.4%
Loss at step 400: 1.293438
Training accuracy: 77.1%
Validation accuracy: 75.0%
Loss at step 500: 1.150579
Training accuracy: 78.0%
Validation accuracy: 75.2%
Loss at step 600: 1.039298
Training accuracy: 78.8%
Validation accuracy: 75.4%
Loss at step 700: 0.950218
Training accuracy: 79.4%
Validation accuracy: 75.8%
Loss at step 800: 0.877612
Training accuracy: 80.3%
Validation accuracy: 76.2%
Test accuracy: 83.6%


In [65]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  learning_rate = 0.5 
  penalty_constant = 1/float(train_subset)
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h) ###
  logits = tf.matmul(lay1_train, weights) + biases #the output for training
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  l2 = loss + penalty_constant*(tf.nn.l2_loss(weights_h)+tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases_h)+tf.nn.l2_loss(biases)+tf.nn.l2_loss(lay1_train))
  # should we also put l2_loss(lay1_train)?
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These lines are not necessary for the training, but will be useful for reporting accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)
  valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights) + biases)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
  test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights) + biases)

In [70]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


KeyboardInterrupt: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [55]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  learning_rate = 0.5
  penalty_constant = 1/float(batch_size*10)
  logits = tf.matmul(tf_train_dataset, weights) + biases #return (train_subset, num_labels)
  #print(logits.get_shape())
  ##penalty= penalty_constant*1/2*(weights)**2 == nn.l2_loss(t) #returns (image_size**2, image_size**2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  l2 = loss + penalty_constant*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases))
  #loss and l2 are not defined until sess.run()
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l2)

  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [56]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.247726
Minibatch accuracy: 11.7%
Validation accuracy: 12.7%
Minibatch loss at step 500: 1.043796
Minibatch accuracy: 78.9%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 0.872749
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.603986
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.595394
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 0.766588
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.682211
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Test accuracy: 88.7%


NN with batches

In [59]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  learning_rate = 0.5 
  penalty_constant = 1/float(batch_size)
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h) ###
  logits = tf.matmul(lay1_train, weights) + biases #the output for training
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  l2 = loss + penalty_constant*(tf.nn.l2_loss(weights_h)+tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases_h)+tf.nn.l2_loss(biases)+tf.nn.l2_loss(lay1_train))
  # should we also put l2_loss(lay1_train)?
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These lines are not necessary for the training, but will be useful for reporting accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)
  valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights) + biases)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
  test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights) + biases)

In [61]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 294.182312
Minibatch accuracy: 16.4%
Validation accuracy: 34.1%
Minibatch loss at step 500: 20.969385
Minibatch accuracy: 77.3%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 9.961599
Minibatch accuracy: 79.7%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 6.915231
Minibatch accuracy: 85.9%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 2.763579
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 4.183051
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 1.921559
Minibatch accuracy: 85.2%
Validation accuracy: 82.1%
Test accuracy: 89.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [81]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  learning_rate = 0.5 
  penalty_constant = 1/float(batch_size)
  lay1_train = tf.nn.relu(tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob = 0.8), weights_h) + biases_h) ###
  logits = tf.matmul(lay1_train, weights) + biases #the output for training
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  l2 = loss + penalty_constant*(tf.nn.l2_loss(weights_h)+tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases_h)+tf.nn.l2_loss(biases)+tf.nn.l2_loss(lay1_train))
  # should we also put l2_loss(lay1_train)?
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These lines are not necessary for the training, but will be useful for reporting accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)
  valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights) + biases)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
  test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights) + biases)

In [85]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 412.691895
Training accuracy: 7.2%
Validation accuracy: 45.5%
Loss at step 100: 27.065226
Training accuracy: 80.1%
Validation accuracy: 80.7%
Loss at step 200: 9.026413
Training accuracy: 86.9%
Validation accuracy: 82.0%
Loss at step 300: 4.829381
Training accuracy: 89.8%
Validation accuracy: 82.4%
Loss at step 400: 4.053327
Training accuracy: 91.0%
Validation accuracy: 82.9%
Loss at step 500: 2.676946
Training accuracy: 92.4%
Validation accuracy: 83.2%
Loss at step 600: 2.381296
Training accuracy: 93.0%
Validation accuracy: 83.4%
Loss at step 700: 1.903017
Training accuracy: 93.8%
Validation accuracy: 83.7%
Loss at step 800: 1.870668
Training accuracy: 94.1%
Validation accuracy: 83.9%
Test accuracy: 90.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [120]:
train_subset = 10000
image_size = 28
num_hidden_nodes2 = 1024
num_hidden_nodes = 200

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes2]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes]))
  biases_2 = tf.Variable(tf.zeros([num_hidden_nodes]))
  #weights_3 = tf.Variable(
  #  tf.truncated_normal([num_hidden_nodes, num_hidden_nodes]))
  #biases_3 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  penalty_constant = 1/float(10000)
  lay1_train = tf.nn.relu(tf.matmul(tf.nn.dropout(tf_train_dataset, keep_prob = 0.8), weights_1) + biases_1) ###
  lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights_2) + biases_2)
  #lay3_train = tf.nn.tanh(tf.matmul(lay2_train, weights_3) + biases_3)
  logits = tf.matmul(lay2_train, weights) + biases #the output for training
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #l2 = loss + penalty_constant*(tf.nn.l2_loss(weights))
  l2 = loss + penalty_constant*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(biases_1)+tf.nn.l2_loss(biases_2)+tf.nn.l2_loss(weights_1))
  # should we also put l2_loss(lay1_train)?
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step = global_step, decay_steps = 300, decay_rate = 0.1)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These lines are not necessary for the training, but will be useful for reporting accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights_2) + biases_2)
  #lay3_valid = tf.nn.tanh(tf.matmul(lay2_valid, weights_3) + biases_3)
  valid_prediction = tf.nn.softmax(tf.matmul(lay2_valid, weights) + biases)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights_2) + biases_2)
  #lay3_test = tf.nn.tanh(tf.matmul(lay2_test, weights_3) + biases_3)
  test_prediction = tf.nn.softmax(tf.matmul(lay2_test, weights) + biases)

In [121]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, l2, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 3188.920898
Training accuracy: 10.5%
Validation accuracy: 10.0%
Loss at step 100: nan
Training accuracy: 9.7%
Validation accuracy: 10.0%


KeyboardInterrupt: 